In [1]:
import itertools as it
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np
import mysql_connector_ssh_rsa

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score, plot_roc_curve, plot_precision_recall_curve, ConfusionMatrixDisplay, confusion_matrix
import test2

In [2]:
df = test2.Data.df

In [9]:
all_var = ['total', 'distance_feed', 'distance_return', 'cargo_count', 'is_individual_calc', 'source_id', 'communication_method', 'payment_method', 'duration', 'distance_out_of_city', 'driver_percent', 'paid_by_cash', 'discount', 'tariff_param_id']
numeric = ['total', 'distance_feed', 'distance_return', 'distance_out_of_city', 'paid_by_cash']
categorial = ['cargo_count', 'is_individual_calc', 'source_id', 'communication_method', 'payment_method',  'duration', 'driver_percent', 'discount', 'tariff_param_id']
# 1, 2, 3, 4, 5

In [10]:
def separation(x):
    numeric_1 = []
    categorial_1 = []
    for i in x:
        if i in numeric:
            numeric_1 += [i]
        if i in categorial:
            categorial_1 += [i]
    return numeric_1, categorial_1, x

In [11]:
def research(numeric_1, categorial_1, x, df):
    df_research = pd.concat([df[['status_id']], df[x]], axis=1)
    df_research = df_research[(df_research.status_id == 'DELETE') | (df_research.status_id == 'DONE')]
    df_research.status_id = df_research.status_id.map({'DONE': 1, 'DELETE': 0})
    if len(categorial_1) == 0:
        df_dummy = df_research
    else:
        df_dummy = pd.get_dummies(df_research, columns=categorial_1, drop_first=True)
    numeric_cols = numeric_1
    cat_cols = list(set(df_dummy.columns) - set(numeric_cols) - {'status_id'})
    cat_cols.sort()
    random_seed = 888
    df_train, df_test = df_dummy, df_dummy
    # Преобразуем возраст в скалярную величину
    if len(numeric_1) > 0:
        scaler = StandardScaler()
        scaler.fit(df_dummy[numeric_cols])
    else:
        scaler = 0

    def get_features_and_target_arrays(df, numeric_cols, cat_cols, scaler):
        if len(numeric_cols) != 0 and len(cat_cols) != 0:
            X_numeric_scaled = scaler.transform(df[numeric_cols])
            X_categorical = df[cat_cols].to_numpy()
            X = np.hstack((X_categorical, X_numeric_scaled))
            y = df['status_id']
        elif len(numeric_cols) == 0 and len(cat_cols) != 0:
            X = df[cat_cols].to_numpy()
            y = df['status_id']
        elif len(cat_cols) == 0 and len(numeric_cols) != 0:
            X = scaler.transform(df[numeric_cols])
            y = df['status_id']
        return X, y

    X, y = get_features_and_target_arrays(df_dummy, numeric_cols, cat_cols, scaler)

    clf = LogisticRegression(penalty='none')
    clf.fit(X, y)

    X_test, y_test = get_features_and_target_arrays(df_test, numeric_cols, cat_cols, scaler)

    test_prob = clf.predict_proba(X_test)[:, 1]
    test_pred = clf.predict(X_test)

    conf_matrix = confusion_matrix(y_test, test_pred)
    return x, conf_matrix

In [12]:
result = pd.DataFrame([], columns=['var', '00', '01', '10', '11'])
for j in range(2, 13):
    for i in it.combinations(all_var, j):
        x = list(i)
        numeric_1, categorial_1, x = separation(x)
        x, conf_matrix = research(numeric_1, categorial_1, x, df)
        x = pd.Series(str(x))
        conf_matrix = list(conf_matrix.flatten())
        conf_matrix = pd.Series(conf_matrix)
        conf_matrix = pd.concat([x, conf_matrix], axis=0, ignore_index=True)
        conf_matrix.index = ['var', '00', '01', '10', '11']
        result = result.append(conf_matrix, ignore_index=True)

In [13]:
result

,var,00,01,10,11
0,"['total', 'distance_feed']",2498,0,1234,0
1,"['total', 'distance_return']",2498,0,1234,0
2,"['total', 'cargo_count']",2462,36,1208,26
3,"['total', 'is_individual_calc']",1942,556,615,619
4,"['total', 'source_id']",2498,0,1234,0
...,...,...,...,...,...
16349,"['distance_feed', 'distance_return', 'cargo_co...",2159,339,405,829
16350,"['distance_feed', 'distance_return', 'cargo_co...",2214,284,595,639
16351,"['distance_feed', 'distance_return', 'is_indiv...",2191,307,455,779
16352,"['distance_feed', 'cargo_count', 'is_individua...",2159,339,414,820


In [14]:
result['error_sum'] = result['01'] + result['10']
result

,var,00,01,10,11,error_sum
0,"['total', 'distance_feed']",2498,0,1234,0,1234
1,"['total', 'distance_return']",2498,0,1234,0,1234
2,"['total', 'cargo_count']",2462,36,1208,26,1244
3,"['total', 'is_individual_calc']",1942,556,615,619,1171
4,"['total', 'source_id']",2498,0,1234,0,1234
...,...,...,...,...,...,...
16349,"['distance_feed', 'distance_return', 'cargo_co...",2159,339,405,829,744
16350,"['distance_feed', 'distance_return', 'cargo_co...",2214,284,595,639,879
16351,"['distance_feed', 'distance_return', 'is_indiv...",2191,307,455,779,762
16352,"['distance_feed', 'cargo_count', 'is_individua...",2159,339,414,820,753


In [17]:
print(np.array(result.loc[16294]))

["['total', 'distance_feed', 'distance_return', 'cargo_count', 'is_individual_calc', 'communication_method', 'payment_method', 'duration', 'distance_out_of_city', 'paid_by_cash', 'discount', 'tariff_param_id']"
 2160 338 403 831 741]
